### IAM Policy Tests with Boto3

This notebook shows how to test IAM policy changes against restrictive IAM Policies  

The key to this policy is to allow EC2 instances that have the `Vendor:Databricks` tag defined.  
There are 2 resources that have this tag, the EBS Volumes and the Instance itself.  
You'll need to pass these tags to the `TagSpecifications=` argument with the correct resources. 

In [ ]:
import boto3 

# set the profile that is defined in the ~/.aws/config file
profile = 'iam-dbx-tests'

session = boto3.Session(profile_name=profile)
ec2=session.client("ec2", region_name='us-east-2')

response = ec2.run_instances(
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/xvda',
            'Ebs': {

                'DeleteOnTermination': True,
                'VolumeSize': 8,
                'VolumeType': 'gp2'
            },
        },
    ],
    ImageId='ami-09246ddb00c7c4fef',
    InstanceType='i3.xlarge',
    MaxCount=1,
    MinCount=1,
    Monitoring={
        'Enabled': False
    },
    KeyName='my-test-key',
    
    SubnetId='subnet-0xdeadbeef',
    
    SecurityGroupIds=[
        'sg-000foo000bar'
    ],
    TagSpecifications=[
              {
                  'ResourceType': 'instance',
                  'Tags': [
                            {'Key': 'Vendor',
                            'Value': 'Databricks'}
                      
                        ]
                },
                {
                  'ResourceType': 'volume',
                  'Tags': [
                            {'Key': 'Vendor',
                            'Value': 'Databricks'}
                        ]
                }
        ]
)


In [ ]:
# print the response from RunInstance
response

### Terminate Instances
Attempt to terminate the instance that was just launched using the same boto3 client defined above. 

In [ ]:
term_resp = ec2.terminate_instances(
    InstanceIds=[
        'i-0xdeadbeef',
    ]
)

In [ ]:
# print the TerminateInstance response
term_resp